# 🏀 EDJBA League Overview

## Key Findings
- **16,955 players** tracked across 13 seasons of Eastern Districts Junior Basketball
- **91,704 stat lines** covering U10 through U18+ age groups
- **40,505 games** recorded with full box scores
- Scoring follows a right-skewed distribution — most players score 2-6 PPG, with a long tail of elite scorers

---

*Data source: PlayHQ GraphQL API (Basketball Victoria) — reverse-engineered, no auth required.*

In [ ]:
%matplotlib inline
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 120

# Connect to the database
DB_PATH = "../data/playhq.db"
conn = sqlite3.connect(DB_PATH)
print("Connected to playhq.db")

## Dataset Scale

Let's start by understanding the sheer volume of data we're working with.

In [ ]:
# Dataset overview
tables = {
    'organisations': 'Basketball organisations in Victoria',
    'seasons': 'Competition seasons (Summer/Winter)',
    'grades': 'Age/skill divisions (e.g., Boys U14 BF)',
    'players': 'Individual players',
    'player_stats': 'Player stat lines (per grade per season)',
    'games': 'Individual game records',
    'teams': 'Team registrations',
    'rounds': 'Competition rounds',
}

print("=" * 55)
print(f"{'Table':<20} {'Rows':>10}   Description")
print("=" * 55)
for table, desc in tables.items():
    count = pd.read_sql(f"SELECT COUNT(*) as n FROM {table}", conn).iloc[0, 0]
    print(f"{table:<20} {count:>10,}   {desc}")

## Player Statistics — Aggregated View

We aggregate each player's stats across all grades and seasons to get career totals.

In [ ]:
# Load aggregated player data
df = pd.read_sql("""
    SELECT
        p.id as player_id,
        p.first_name || ' ' || p.last_name as player_name,
        SUM(ps.games_played) as games_played,
        SUM(ps.total_points) as total_points,
        SUM(ps.one_point) as free_throws_made,
        SUM(ps.two_point) as two_pt_made,
        SUM(ps.three_point) as three_pt_made,
        SUM(ps.total_fouls) as total_fouls,
        COUNT(DISTINCT ps.grade_id) as seasons_played
    FROM player_stats ps
    JOIN players p ON p.id = ps.player_id
    GROUP BY p.id
    HAVING SUM(ps.games_played) > 0
""", conn)

df["ppg"] = df["total_points"] / df["games_played"]
df["fpg"] = df["total_fouls"] / df["games_played"]

print(f"Players with game data: {len(df):,}")
print(f"Total games played:     {df['games_played'].sum():,.0f}")
print(f"Total points scored:    {df['total_points'].sum():,.0f}")
print(f"\nPPG Summary (all players):")
df[["games_played", "total_points", "ppg", "fpg"]].describe().round(2)

## Scoring Distributions

The PPG distribution is heavily right-skewed — junior basketball has many low-volume scorers and a handful of dominant players.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle("EDJBA Player Statistics Overview", fontsize=16, fontweight="bold", y=1.02)

# PPG distribution
ax = axes[0, 0]
active = df[df["ppg"] > 0]
ax.hist(active["ppg"], bins=60, color="#1976D2", edgecolor="white", alpha=0.85)
ax.axvline(active["ppg"].median(), color="#D32F2F", ls="--", lw=2, label=f'Median: {active["ppg"].median():.1f}')
ax.axvline(active["ppg"].mean(), color="#FF9800", ls="--", lw=2, label=f'Mean: {active["ppg"].mean():.1f}')
ax.set_title("Points Per Game Distribution", fontweight="bold")
ax.set_xlabel("PPG"); ax.set_ylabel("Players"); ax.legend()

# FPG distribution
ax = axes[0, 1]
ax.hist(df["fpg"], bins=40, color="#FF9800", edgecolor="white", alpha=0.85)
ax.axvline(df["fpg"].median(), color="#D32F2F", ls="--", lw=2, label=f'Median: {df["fpg"].median():.1f}')
ax.set_title("Fouls Per Game Distribution", fontweight="bold")
ax.set_xlabel("FPG"); ax.set_ylabel("Players"); ax.legend()

# Scoring breakdown — top 50
ax = axes[1, 0]
top50 = df.nlargest(50, "total_points")
x = range(len(top50))
ax.bar(x, top50["free_throws_made"], label="Free Throws", color="#4CAF50")
ax.bar(x, top50["two_pt_made"] * 2, bottom=top50["free_throws_made"], label="2PT Points", color="#1976D2")
ax.bar(x, top50["three_pt_made"] * 3,
       bottom=top50["free_throws_made"] + top50["two_pt_made"] * 2,
       label="3PT Points", color="#D32F2F")
ax.set_title("Scoring Breakdown — Top 50 Career Scorers", fontweight="bold")
ax.set_xlabel("Player Rank"); ax.set_ylabel("Points"); ax.legend(fontsize=9)

# PPG vs FPG
ax = axes[1, 1]
sample = df[df["games_played"] >= 5].sample(min(2000, len(df[df["games_played"] >= 5])), random_state=42)
scatter = ax.scatter(sample["ppg"], sample["fpg"], alpha=0.3, s=12, c=sample["games_played"],
                     cmap="viridis", edgecolors="none")
plt.colorbar(scatter, ax=ax, label="Games Played")
ax.set_title("PPG vs FPG (5+ games)", fontweight="bold")
ax.set_xlabel("Points Per Game"); ax.set_ylabel("Fouls Per Game")

plt.tight_layout()
plt.savefig("../assets/league_overview.png", dpi=150, bbox_inches="tight")
plt.show()

## Top 20 Scorers (min 5 games)

In [ ]:
top20 = (df[df["games_played"] >= 5]
         .nlargest(20, "ppg")
         [["player_name", "games_played", "total_points", "ppg", "fpg", "two_pt_made", "three_pt_made"]]
         .round(2))
top20.index = range(1, 21)
top20.columns = ["Player", "GP", "PTS", "PPG", "FPG", "2PM", "3PM"]
top20

## Season Coverage

How many seasons of data do we have, and how active is each one?

In [ ]:
seasons = pd.read_sql("""
    SELECT s.name as season, COUNT(DISTINCT g.id) as grades,
           COUNT(DISTINCT ps.player_id) as players,
           SUM(ps.games_played) as total_gp
    FROM seasons s
    JOIN grades g ON g.season_id = s.id
    LEFT JOIN player_stats ps ON ps.grade_id = g.id
    GROUP BY s.id
    ORDER BY s.start_date
""", conn)
seasons

In [ ]:
conn.close()